In [1]:
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.datasets.mnist import load_data
import numpy as np
import time

In [2]:
# Loading and formatting the data 
(x_train, y_train), (x_test,y_test) = load_data()
x_train = np.expand_dims(x_train, axis=3)/255.0
x_test = np.expand_dims(x_test, axis=3)/255.0
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [3]:
def create_model():
    input_x = Input((28,28,1))
    
    x = Conv2D(filters=32, kernel_size=3, activation="relu")(input_x)
    x = AvgPool2D()(x)
    x = Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = AvgPool2D()(x)
    x = Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dense(10, activation="sigmoid")(x)
    model = Model(inputs=input_x, outputs=x)
    model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics="accuracy")
    return model

In [4]:
create_model().summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
average_pooling2d (AveragePo (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 32)          9248      
_________________________________________________________________
flatten (Flatten)            (None, 288)               0     

In [5]:
num_models = 3

for i in range(num_models):
    model = create_model()
    # on GPU using CUDA
    t_start = time.time()
    model.fit(x_train, y_train, batch_size=512, shuffle=True, epochs=5, verbose=0)
    t_end = time.time()
    loss, acc = model.evaluate(x_test,y_test, verbose=0)
    print("Model {}: loss={:0.4f}, accuracy={:0.3f}, training_time={:0.5f}".format(i+1, loss, acc*100, t_end-t_start))

Model 1: loss=0.0666, loss=97.930
Model 2: loss=0.0607, loss=98.110
Model 3: loss=0.0520, loss=98.330


## Fixing the seed

In [7]:
num_models = 3

for i in range(num_models):
    # fixing the seed
    tf.random.set_seed(42)
    np.random.seed(1234) # optional, only necessary if using numpy random functions
    
    model = create_model()
    # on GPU using CUDA
    t_start = time.time()
    model.fit(x_train, y_train, batch_size=512, shuffle=True, epochs=5, verbose=0)
    t_end = time.time()
    loss, acc = model.evaluate(x_test,y_test, verbose=0)
    print("Model {}: loss={:0.4f}, accuracy={:0.3f}, training_time={:0.5f}".format(i+1, loss, acc*100, t_end-t_start))

ERROR! Session/line number was not unique in database. History logging moved to new session 1492
Model 1: loss=0.0604, accuracy=98.050, training_time=2.93664
Model 2: loss=0.0593, accuracy=98.170, training_time=2.82109
Model 3: loss=0.0613, accuracy=98.090, training_time=3.34918
